# Understand the nature of the target variable:

Target Variable: Trip_Price (continuous)

Since the target is continuous, this is a regression problem.

Select appropriate models:
Since this is a regression problem, we can try a few common models like:

Linear Regression: A simple yet powerful model for continuous variables.

Random Forest Regressor: A more powerful, non-linear model that can capture complex relationships.

Gradient Boosting Regressor: Another ensemble model that is typically very strong for regression tasks.

Apply Hyperparameter Tuning:
For each model, we'll perform hyperparameter tuning to improve their performance using GridSearchCV or RandomizedSearchCV.

Evaluation Metrics:
We'll use regression-specific metrics to evaluate the models:

Mean Absolute Error (MAE)

Mean Squared Error (MSE)

R-squared (R²)

In [31]:
#imports libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [32]:
# Load data
df = pd.read_csv('taxi_trip_pricing.csv')
df.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [33]:
# Check data types
print("\nData types of columns:")
print(df.dtypes)


Data types of columns:
Trip_Distance_km         float64
Time_of_Day               object
Day_of_Week               object
Passenger_Count          float64
Traffic_Conditions        object
Weather                   object
Base_Fare                float64
Per_Km_Rate              float64
Per_Minute_Rate          float64
Trip_Duration_Minutes    float64
Trip_Price               float64
dtype: object


In [34]:
df.describe()

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
count,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,951.000000
mean,27.070547,2.476842,3.502989,1.233316,0.292916,62.118116,56.874773
std,19.905300,1.102249,0.870162,0.429816,0.115592,32.154406,40.469791
min,1.230000,1.000000,2.010000,0.500000,0.100000,5.010000,6.126900
25%,12.632500,1.250000,2.730000,0.860000,0.190000,35.882500,33.742650
50%,25.830000,2.000000,3.520000,1.220000,0.290000,61.860000,50.074500
75%,38.405000,3.000000,4.260000,1.610000,0.390000,89.055000,69.099350
max,146.067047,4.000000,5.000000,2.000000,0.500000,119.840000,332.043689


In [35]:
#Create dummy features for categorical variables
categorical_cols = ['Time_of_Day', 'Day_of_Week', 'Traffic_Conditions', 'Weather']

# Apply one-hot encoding to categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [36]:
# Define target and features
target = 'Trip_Price'
X = df_encoded.drop(columns=[target])
y = df_encoded[target]

# Identify numeric columns in X
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform numeric features
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])


In [49]:
print(X.isnull().sum())

Trip_Distance_km             50
Passenger_Count              50
Base_Fare                    50
Per_Km_Rate                  50
Per_Minute_Rate              50
Trip_Duration_Minutes        50
Time_of_Day_Evening           0
Time_of_Day_Morning           0
Time_of_Day_Night             0
Day_of_Week_Weekend           0
Traffic_Conditions_Low        0
Traffic_Conditions_Medium     0
Weather_Rain                  0
Weather_Snow                  0
dtype: int64


In [55]:
# Define target and features
target = 'Trip_Price'
X = df_encoded.drop(columns=[target])
y = df_encoded[target]

# Handle missing values (Imputation)
# Apply SimpleImputer to impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Convert the imputed array back to a DataFrame to maintain column names
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# Split data into training and testing sets (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Model 1: Linear Regression

In [58]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict and evaluate
lr_pred = lr_model.predict(X_test)
lr_mae = mean_absolute_error(y_test, lr_pred)
lr_mse = mean_squared_error(y_test, lr_pred)
lr_r2 = r2_score(y_test, lr_pred)

print("Linear Regression Results:")
print(f"MAE: {lr_mae:.4f}")
print(f"MSE: {lr_mse:.4f}")
print(f"R-squared: {lr_r2:.4f}")


ValueError: Input y contains NaN.

# Model 2: Random Forest Regressor

In [45]:
rf_model = RandomForestRegressor(random_state=42)

# Hyperparameter tuning with GridSearchCV
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(X_train, y_train)

# Best model
rf_best_model = rf_grid_search.best_estimator_

# Predict and evaluate
rf_pred = rf_best_model.predict(X_test)
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

ValueError: 
All the 180 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 348, in fit
    X, y = self._validate_data(
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\base.py", line 622, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\utils

# Model 3: Gradient Boosting Regressor

In [46]:
gb_model = GradientBoostingRegressor(random_state=42)

# Hyperparameter tuning with GridSearchCV
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

gb_grid_search = GridSearchCV(estimator=gb_model, param_grid=gb_param_grid, cv=5, scoring='neg_mean_squared_error')
gb_grid_search.fit(X_train, y_train)

# Best model
gb_best_model = gb_grid_search.best_estimator_

# Predict and evaluate
gb_pred = gb_best_model.predict(X_test)
gb_mae = mean_absolute_error(y_test, gb_pred)
gb_mse = mean_squared_error(y_test, gb_pred)
gb_r2 = r2_score(y_test, gb_pred)

ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 416, in fit
    X, y = self._validate_data(
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\base.py", line 622, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\dellm\anaconda3\lib\site-packages\sklearn\utils\val

# Model Evaluation

In [47]:
print("Linear Regression Performance:")
print(f"MAE: {lr_mae}, MSE: {lr_mse}, R²: {lr_r2}")

print("\nRandom Forest Performance:")
print(f"MAE: {rf_mae}, MSE: {rf_mse}, R²: {rf_r2}")

print("\nGradient Boosting Performance:")
print(f"MAE: {gb_mae}, MSE: {gb_mse}, R²: {gb_r2}")

Linear Regression Performance:


NameError: name 'lr_mae' is not defined

# Model Comparison: Choose the Best

In [48]:
models_performance = {
    'Linear Regression': [lr_mae, lr_mse, lr_r2],
    'Random Forest': [rf_mae, rf_mse, rf_r2],
    'Gradient Boosting': [gb_mae, gb_mse, gb_r2]
}

# Convert to DataFrame for better display
performance_df = pd.DataFrame(models_performance, index=["MAE", "MSE", "R²"]).T
print("\nModel Comparison:")
print(performance_df)

# Identifying the best model based on R² (higher is better)
best_model = performance_df['R²'].idxmax()
print(f"\nThe best model based on R² score is: {best_model}")

NameError: name 'lr_mae' is not defined